In [12]:
# training_images
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [13]:
import os
os.listdir("training_images")

['bleeding', 'non_bleeding']

In [14]:
class_names = sorted(os.listdir("training_images"))

In [15]:
data = []
labels = []
for class_idx, class_name in enumerate(class_names):
    class_dir = os.path.join("training_images", class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        data.append(img_array)
        labels.append(class_idx)

In [16]:
data = np.array(data, dtype=np.uint8)
labels = np.array(labels)

In [17]:
data.shape

(2618, 224, 224, 3)

In [18]:
from tensorflow.keras.utils import to_categorical
num_classes = len(class_names)
labels_encoded = to_categorical(labels, num_classes)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.3, random_state=42)

In [20]:
input_shape = (224,224,3)

In [21]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

In [22]:
from tensorflow.keras.layers import GlobalAveragePooling2D
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='sigmoid')(x)

In [23]:
model = Model(inputs=base_model.input, outputs=predictions)

In [24]:
model.compile(optimizer=Adam(learning_rate=1e-2), loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(X_train, y_train, batch_size = 32, epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
58/58 [==============================] - 172s 3s/step - loss: 0.4263 - accuracy: 0.8908 - val_loss: 0.0595 - val_accuracy: 0.9784
Epoch 2/10
58/58 [==============================] - 171s 3s/step - loss: 0.0874 - accuracy: 0.9689 - val_loss: 0.0345 - val_accuracy: 0.9835
Epoch 3/10
58/58 [==============================] - 178s 3s/step - loss: 0.0807 - accuracy: 0.9727 - val_loss: 0.0398 - val_accuracy: 0.9873
Epoch 4/10
58/58 [==============================] - 180s 3s/step - loss: 0.0538 - accuracy: 0.9831 - val_loss: 0.0371 - val_accuracy: 0.9885
Epoch 5/10
58/58 [==============================] - 183s 3s/step - loss: 0.0290 - accuracy: 0.9913 - val_loss: 0.0401 - val_accuracy: 0.9885
Epoch 6/10
58/58 [==============================] - 184s 3s/step - loss: 0.0214 - accuracy: 0.9945 - val_loss: 0.0371 - val_accuracy: 0.9860
Epoch 7/10
58/58 [==============================] - 153s 3s/step - loss: 0.0308 - accuracy: 0.9918 - val_loss: 0.0323 - val_accuracy: 0.9949
Epoch 8/10
58

In [26]:
model.evaluate(X_test,y_test)

25/25 [==============================] - 45s 2s/step - loss: 0.0795 - accuracy: 0.9758


[0.07951617985963821, 0.9758269786834717]

In [ ]:
model.save("efficientnet.h5")

In [27]:
y_pred = model.predict(X_test)

25/25 [==============================] - 48s 2s/step


In [28]:
y_pred.shape

(786, 2)

In [29]:
for i in range(y_pred.shape[0]):
    max_value = np.max(y_pred[i])
    y_pred[i] = np.where(y_pred[i] == max_value, 1, 0)

In [25]:
y_pred

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [20]:
type(class_names)

list

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98       393
           1       0.96      1.00      0.98       393

   micro avg       0.98      0.98      0.98       786
   macro avg       0.98      0.98      0.98       786
weighted avg       0.98      0.98      0.98       786
 samples avg       0.98      0.98      0.98       786



In [10]:
data = []
labels = []
for class_idx, class_name in enumerate(class_names):
    class_dir = os.path.join("training_images", class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        data.append(img_array)
        labels.append(class_idx)

KeyboardInterrupt: 

In [ ]:
os.path.join("Auto-WCEBleedGen Challenge Test Dataset/Test_Dataset_2", i)

In [33]:
# E:/GI bleeding detection/Auto-WCEBleedGen Challenge Test Dataset_mrin/Testing_images
data = []
files = []
for i in os.listdir("E:/GI bleeding detection/Auto-WCEBleedGen Challenge Test Dataset_mrin/Testing_images"):
    img = load_img(os.path.join("E:/GI bleeding detection/Auto-WCEBleedGen Challenge Test Dataset_mrin/Testing_images", i), target_size=(224,224))
    img_array = img_to_array(img)
    data.append(img_array)
    files.append(i)

In [34]:
files[8]

'A0008.png'

In [35]:
data = np.array(data, dtype=np.uint8)

In [36]:
y_pred = model.predict(data)

18/18 [==============================] - 24s 1s/step


In [38]:
for i in range(y_pred.shape[0]):
    max_value = np.max(y_pred[i])
    y_pred[i] = np.where(y_pred[i] == max_value, 1, 0)

In [40]:
import os
import numpy as np
import shutil

In [42]:
output_dir = "E:/GI bleeding detection/Auto-WCEBleedGen Challenge Test Dataset_mrin/tested"
bleeding_dir = os.path.join(output_dir, "bleeding")
non_bleeding_dir = os.path.join(output_dir, "non_bleeding")

# Create the main output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create subdirectories for bleeding and non-bleeding images
if not os.path.exists(bleeding_dir):
    os.makedirs(bleeding_dir)
if not os.path.exists(non_bleeding_dir):
    os.makedirs(non_bleeding_dir)


In [49]:
import pandas as pd
df = pd.DataFrame()

In [50]:
files[0]

'A0000.png'

In [51]:
lbl = []
for i, label in enumerate(y_pred):
    file_path = files[i]  # Get the file path for the current image

    # Determine the label and destination directory
    
    if label[0] == 1:
        destination_dir = bleeding_dir
        lbl.append("Bleeding")
        
    elif label[1] == 1:
        destination_dir = non_bleeding_dir
        lbl.append("Non Bleeding")
    else:
        continue  # Skip rows without a valid label

    # Extract the image file name from the file path
    file_path = os.path.join("E:/GI bleeding detection/Auto-WCEBleedGen Challenge Test Dataset_mrin/Testing_images", files[i])
    file_name = os.path.basename(file_path)

    # Construct the destination file path
    destination_file_path = os.path.join(destination_dir, file_name)

    # Copy the image to the appropriate directory
    shutil.copy(file_path, destination_file_path)


In [52]:
files

['A0000.png',
 'A0001.png',
 'A0002.png',
 'A0003.png',
 'A0004.png',
 'A0005.png',
 'A0006.png',
 'A0007.png',
 'A0008.png',
 'A0009.png',
 'A0010.png',
 'A0011.png',
 'A0012.png',
 'A0013.png',
 'A0014.png',
 'A0016.png',
 'A0017.png',
 'A0018.png',
 'A0019.png',
 'A0020.png',
 'A0021.png',
 'A0022.png',
 'A0023.png',
 'A0024.png',
 'A0025.png',
 'A0026.png',
 'A0027.png',
 'A0028.png',
 'A0029.png',
 'A0030.png',
 'A0031.png',
 'A0032.png',
 'A0033.png',
 'A0034.png',
 'A0035.png',
 'A0036.png',
 'A0037.png',
 'A0038.png',
 'A0039.png',
 'A0040.png',
 'A0041.png',
 'A0042.png',
 'A0043.png',
 'A0044.png',
 'A0045.png',
 'A0046.png',
 'A0047.png',
 'A0048.png',
 'A0049.png',
 'A0050.png',
 'A0051.png',
 'A0052.png',
 'A0053.png',
 'A0054.png',
 'A0055.png',
 'A0056.png',
 'A0057.png',
 'A0058.png',
 'A0059.png',
 'A0060.png',
 'A0061.png',
 'A0062.png',
 'A0063.png',
 'A0064.png',
 'A0065.png',
 'A0066.png',
 'A0067.png',
 'A0068.png',
 'A0069.png',
 'A0070.png',
 'A0071.png',
 'A007

In [53]:
df['files'] = files
df['Label'] = lbl

In [56]:
df["Label"].value_counts()

Non Bleeding    380
Bleeding        184
Name: Label, dtype: int64

In [58]:
df.to_csv("results.csv")